```sql
-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN   VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATIONFOR MORE INFORMATION.
```

# **INTRODUCTIONS**

---

This pybook is for the old flow for the file: 'https://lcps.nu/wp-content/uploads/covid-19-datafeed-oud-v1.csv'

The source code is separated into multiple sections:

1. **[Tables](#tables)**
2. **[Stored Procedures](#stored-procedures)**
3. **[Views](#views)**
4. **[Datatino Configurations](#datatino-configurations)**

# **DEPENDENCIES**

---

```json
{
    "depends-on": [
        "src/utils/conversions.ipynb",
        "src/utils/schemas.ipynb",
        "src/utils/protos.ipynb"
    ]
}
```

# **TABLES**
---

### STAGINGS

In [ ]:
-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATION FOR MORE INFORMATION.

SET ANSI_NULLS ON
GO

SET QUOTED_IDENTIFIER ON
GO

-- 1) CREATE TABLE(S).....
IF NOT EXISTS(SELECT * FROM SYS.TABLES WHERE [OBJECT_ID] = OBJECT_ID(N'[VWSSTAGE].[LNAZ_HOSPITAL_BED_OCCUPANCY]'))
BEGIN
    CREATE TABLE [VWSSTAGE].[LNAZ_HOSPITAL_BED_OCCUPANCY](
        [ID] [INT] PRIMARY KEY IDENTITY(1,1),
        [DATUM] [VARCHAR](100) NULL,
        [IC_BEDDEN_COVID] [VARCHAR](100) NULL,
        [IC_BEDDEN_NON_COVID] [VARCHAR](100) NULL,
        [KLINIEK_BEDDEN] [VARCHAR](100) NULL,
        [DATE_LAST_INSERTED] [DATETIME] NULL DEFAULT (GETDATE()),
        [IC_NIEUWE_OPNAMES_COVID] [VARCHAR](100) NULL,
        [KLINIEK_NIEUWE_OPNAMES_COVID] [VARCHAR](100) NULL
        );
END


### INTERMEDIATES

In [ ]:
-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATION FOR MORE INFORMATION.

SET ANSI_NULLS ON
GO

SET QUOTED_IDENTIFIER ON
GO

-- 1) CREATE TABLE(S).....
IF NOT EXISTS(SELECT * FROM SYS.TABLES WHERE [OBJECT_ID] = OBJECT_ID(N'[VWSINTER].[LNAZ_HOSPITAL_BED_OCCUPANCY]'))
BEGIN
    CREATE TABLE [VWSINTER].[LNAZ_HOSPITAL_BED_OCCUPANCY](
        [ID] [INT] PRIMARY KEY IDENTITY(1,1),
        [DATUM] [DATETIME] NULL,
        [IC_BEDDEN_COVID] [INT] NULL,
        [IC_BEDDEN_NON_COVID] [INT] NULL,
        [KLINIEK_BEDDEN] [INT] NULL,
        [DATE_LAST_INSERTED] [DATETIME] NULL DEFAULT (GETDATE()),
        [IC_NIEUWE_OPNAMES_COVID] [INT] NULL,
        [KLINIEK_NIEUWE_OPNAMES_COVID] [INT] NULL
        );
END
ELSE
BEGIN
    IF NOT EXISTS (SELECT * FROM SYS.COLUMNS 
        WHERE [NAME] IN (N'IC_NIEUWE_OPNAMES_COVID')
        AND [OBJECT_ID] = OBJECT_ID(N'[VWSINTER].[LNAZ_HOSPITAL_BED_OCCUPANCY]'))
            BEGIN
                ALTER TABLE [VWSINTER].[LNAZ_HOSPITAL_BED_OCCUPANCY]
                ADD 
                    [IC_NIEUWE_OPNAMES_COVID] [INT] NULL;
            END
    IF NOT EXISTS (SELECT * FROM SYS.COLUMNS 
        WHERE [NAME] IN (N'KLINIEK_NIEUWE_OPNAMES_COVID')
        AND [OBJECT_ID] = OBJECT_ID(N'[VWSINTER].[LNAZ_HOSPITAL_BED_OCCUPANCY]'))
            BEGIN
                ALTER TABLE [VWSINTER].[LNAZ_HOSPITAL_BED_OCCUPANCY]
                ADD 
                    [KLINIEK_NIEUWE_OPNAMES_COVID] [INT] NULL;
            END
END
GO


### DESTINATIONS

In [ ]:
-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATION FOR MORE INFORMATION.

SET ANSI_NULLS ON
GO

SET QUOTED_IDENTIFIER ON
GO

-- 1) CREATE TABLE(S).....
IF NOT EXISTS(SELECT * FROM SYS.TABLES WHERE [OBJECT_ID] = OBJECT_ID(N'[VWSDEST].[HOSPITAL_BED_OCCUPANCY]'))
BEGIN
    CREATE TABLE [VWSDEST].[HOSPITAL_BED_OCCUPANCY](
        [ID] [INT] PRIMARY KEY IDENTITY(1,1),
        [DATE_OF_REPORT] [DATETIME] NULL,
        [DATE_OF_REPORT_UNIX] [BIGINT] NULL,
        [IC_BEDS_OCCUPIED_COVID] [INT] NULL,
        [IC_BEDS_OCCUPIED_NON_COVID] [INT] NULL,
        [IC_BEDS_OCCUPIED_COVID_PERCENTAGE] [DECIMAL](16, 2) NULL,
        [NON_IC_BEDS_OCCUPIED_COVID] [INT] NULL,
        [DATE_LAST_INSERTED] [DATETIME] NULL DEFAULT (GETDATE()),
        [IC_INFLUX_COVID] [INT] NULL,
        [NON_IC_INFLUX_COVID] [INT] NULL
        );
END
ELSE
BEGIN
    IF NOT EXISTS (SELECT * FROM SYS.COLUMNS 
        WHERE [NAME] IN (N'IC_INFLUX_COVID')
        AND [OBJECT_ID] = OBJECT_ID(N'[VWSDEST].[HOSPITAL_BED_OCCUPANCY]'))
            BEGIN
                ALTER TABLE [VWSDEST].[HOSPITAL_BED_OCCUPANCY]
                ADD 
                    [IC_INFLUX_COVID] [INT] NULL;
            END
    IF NOT EXISTS (SELECT * FROM SYS.COLUMNS 
        WHERE [NAME] IN (N'NON_IC_INFLUX_COVID')
        AND [OBJECT_ID] = OBJECT_ID(N'[VWSDEST].[HOSPITAL_BED_OCCUPANCY]'))
            BEGIN
                ALTER TABLE [VWSDEST].[HOSPITAL_BED_OCCUPANCY]
                ADD 
                    [NON_IC_INFLUX_COVID] [INT] NULL;
            END
END
GO


# **STORED PROCEDURES**
---

### STAGING &rarr; INTERMEDIATE MAPPING

In [ ]:
-- Copyright (c) 2020 De Staat der Nederlanden, Ministerie van   Volksgezondheid, Welzijn en Sport. 
-- Licensed under the EUROPEAN UNION PUBLIC LICENCE v. 1.2 - see https://github.com/minvws/nl-contact-tracing-app-coordinationfor more information.

-- Move national data from staging to intermediate table.

-- 1) CREATE STORE PROCEDURE(S) STAGE -> INTER.....
CREATE OR ALTER PROCEDURE [dbo].[SP_LNAZ_HOSPITAL_BED_OCCUPANCY_INTER]
AS
BEGIN
    INSERT INTO VWSINTER.LNAZ_HOSPITAL_BED_OCCUPANCY
        (   [DATUM],
            [IC_BEDDEN_COVID],
            [IC_BEDDEN_NON_COVID],
            [KLINIEK_BEDDEN],
            [IC_NIEUWE_OPNAMES_COVID],
            [KLINIEK_NIEUWE_OPNAMES_COVID]
        )    SELECT
    --- conversion of date in dd-mm-yyyy format ---
        CONVERT(DATETIME, [DATUM], 105),
    -- allow empty values and make them null
        IIF([IC_BEDDEN_COVID] IN ('', 'NA'), NULL, TRIM([IC_BEDDEN_COVID])),
        IIF([IC_BEDDEN_NON_COVID] IN ('', 'NA'), NULL, TRIM([IC_BEDDEN_NON_COVID])),
        IIF([KLINIEK_BEDDEN] IN ('', 'NA'), NULL, TRIM([KLINIEK_BEDDEN])),
        IIF([IC_NIEUWE_OPNAMES_COVID] IN ('', 'NA'), NULL, TRIM([IC_NIEUWE_OPNAMES_COVID])),
        IIF([KLINIEK_NIEUWE_OPNAMES_COVID] IN ('', 'NA'), NULL, TRIM([KLINIEK_NIEUWE_OPNAMES_COVID]))
    FROM
       VWSSTAGE.LNAZ_HOSPITAL_BED_OCCUPANCY
    WHERE DATE_LAST_INSERTED = (SELECT MAX(DATE_LAST_INSERTED) FROM VWSSTAGE.LNAZ_HOSPITAL_BED_OCCUPANCY)
END;
GO

### INTERMEDIATE &rarr; DESTINATION MAPPING

In [ ]:
-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATION FOR MORE INFORMATION.

-- 1) CREATE STORE PROCEDURE(S) INTER -> DEST.....
CREATE OR ALTER PROCEDURE [dbo].[SP_HOSPITAL_BED_OCCUPANCY]
AS
BEGIN
    INSERT INTO VWSDEST.HOSPITAL_BED_OCCUPANCY
        (   [DATE_OF_REPORT],
	        [DATE_OF_REPORT_UNIX],
            [IC_BEDS_OCCUPIED_COVID],
            [IC_BEDS_OCCUPIED_NON_COVID],
            [IC_BEDS_OCCUPIED_COVID_PERCENTAGE],
            [NON_IC_BEDS_OCCUPIED_COVID],
            [IC_INFLUX_COVID],
            [NON_IC_INFLUX_COVID]
        )
    SELECT
        [DATUM] AS DATE_OF_REPORT,
        dbo.CONVERT_DATETIME_TO_UNIX(DATUM) AS DATE_OF_REPORT_UNIX,
        [IC_BEDDEN_COVID] AS IC_BEDS_OCCUPIED_COVID,
        [IC_BEDDEN_NON_COVID] AS IC_BEDS_OCCUPIED_NON_COVID,
        --If either IC_BEDDEN_COVID or IC_BEDDEN_NON_COVID equals NULL, NULL is the result of the line below
        ROUND((CAST([IC_BEDDEN_COVID] as DECIMAL)/([IC_BEDDEN_NON_COVID]+[IC_BEDDEN_COVID]) * 100.0),2) AS IC_BEDS_OCCUPIED_COVID_PERCENTAGE,
        [KLINIEK_BEDDEN] AS NON_IC_BEDS_OCCUPIED_COVID,
        [IC_NIEUWE_OPNAMES_COVID],
        [KLINIEK_NIEUWE_OPNAMES_COVID]
    FROM
       VWSINTER.LNAZ_HOSPITAL_BED_OCCUPANCY
    WHERE DATE_LAST_INSERTED = (SELECT MAX(DATE_LAST_INSERTED) FROM VWSINTER.LNAZ_HOSPITAL_BED_OCCUPANCY)
END;
GO


# **VIEWS**
---

### VIEWS - HOSPITAL

In [ ]:
-- Copyright (c) 2020 De Staat der Nederlanden, Ministerie van   Volksgezondheid, Welzijn en Sport. 
-- Licensed under the EUROPEAN UNION PUBLIC LICENCE v. 1.2 - see https://github.com/minvws/nl-contact-tracing-app-coordinationfor more information.

CREATE OR ALTER VIEW [VWSDEST].[V_NON_IC_BED_OCCUPANCY] AS
SELECT 
    [DATE_OF_REPORT_UNIX] AS DATE_UNIX,
    [NON_IC_BEDS_OCCUPIED_COVID] as beds_occupied_covid,
    [NON_IC_INFLUX_COVID] AS [INFLUX_COVID_PATIENTS],
    dbo.CONVERT_DATETIME_TO_UNIX(DATE_LAST_INSERTED)  AS DATE_OF_INSERTION_UNIX
FROM [VWSDEST].[HOSPITAL_BED_OCCUPANCY]
WHERE DATE_LAST_INSERTED = (SELECT MAX(DATE_LAST_INSERTED)
                            FROM [VWSDEST].[HOSPITAL_BED_OCCUPANCY]);
GO

In [ ]:
-- Copyright (c) 2020 De Staat der Nederlanden, Ministerie van   Volksgezondheid, Welzijn en Sport. 
-- Licensed under the EUROPEAN UNION PUBLIC LICENCE v. 1.2 - see https://github.com/minvws/nl-contact-tracing-app-coordinationfor more information.

CREATE OR ALTER VIEW [VWSDEST].[V_DIFFERENCE_HOSPITAL_BEDS_OCCUPIED__COVID_OCCUPIED] AS

WITH LAST_DATE_OF_REPORT AS (
	SELECT
		CAST(MAX(DATE_OF_REPORT) as datetime) AS [LAST_DATE_OF_REPORT]
	FROM [VWSDEST].[HOSPITAL_BED_OCCUPANCY]
	WHERE DATE_OF_REPORT >=  '2020-03-16 00:00:00.000'
		AND DATE_LAST_INSERTED = (SELECT MAX(DATE_LAST_INSERTED) FROM [VWSDEST].[HOSPITAL_BED_OCCUPANCY])
),

BASE_CTE AS (
SELECT 
	DATE_OF_REPORT,
	DATE_OF_REPORT_UNIX AS [NEW_DATE_OF_REPORT_UNIX],
	LAG([DATE_OF_REPORT_UNIX], 1, NULL) OVER (ORDER BY [DATE_LAST_INSERTED], [DATE_OF_REPORT_UNIX]) AS [OLD_DATE_OF_REPORT_UNIX],
	LAG([NON_IC_BEDS_OCCUPIED_COVID], 1, NULL) OVER (ORDER BY [DATE_LAST_INSERTED], [DATE_OF_REPORT_UNIX]) AS [OLD_VALUE],
	[NON_IC_BEDS_OCCUPIED_COVID] -
		LAG([NON_IC_BEDS_OCCUPIED_COVID], 1, NULL) OVER (ORDER BY [DATE_LAST_INSERTED], [DATE_OF_REPORT_UNIX]) AS [DIFFERENCE]
--  dbo.CONVERT_DATETIME_TO_UNIX(DATE_LAST_INSERTED) AS DATE_OF_INSERTION_UNIX
FROM [VWSDEST].[HOSPITAL_BED_OCCUPANCY]
WHERE DATE_OF_REPORT >=  '2020-03-02 00:00:00.000'
AND DATE_LAST_INSERTED = (SELECT max(DATE_LAST_INSERTED) 
                            FROM [VWSDEST].[HOSPITAL_BED_OCCUPANCY])

)

SELECT DATE_OF_REPORT
	  ,NEW_DATE_OF_REPORT_UNIX AS NEW_DATE_UNIX
	  ,OLD_DATE_OF_REPORT_UNIX AS OLD_DATE_UNIX
	  ,CASE WHEN OLD_VALUE IS NULL THEN 0 ELSE OLD_VALUE END AS OLD_VALUE
	  ,CASE WHEN [DIFFERENCE] IS NULL THEN 0 ELSE [DIFFERENCE] END AS [DIFFERENCE]
FROM BASE_CTE T1
WHERE DATE_OF_REPORT = (SELECT LAST_DATE_OF_REPORT FROM LAST_DATE_OF_REPORT)
GO


### VIEWS - IC

In [ ]:
-- Copyright (c) 2020 De Staat der Nederlanden, Ministerie van   Volksgezondheid, Welzijn en Sport. 
-- Licensed under the EUROPEAN UNION PUBLIC LICENCE v. 1.2 - see https://github.com/minvws/nl-contact-tracing-app-coordinationfor more information.

CREATE OR ALTER VIEW [VWSDEST].[V_IC_BED_OCCUPANCY] AS
SELECT 
    [DATE_OF_REPORT_UNIX] AS DATE_UNIX,
    [IC_BEDS_OCCUPIED_COVID] as BEDS_OCCUPIED_COVID,
    [IC_BEDS_OCCUPIED_NON_COVID] as BEDS_OCCUPIED_NON_COVID,
    [IC_BEDS_OCCUPIED_COVID_PERCENTAGE] as BEDS_OCCUPIED_COVID_PERCENTAGE,
    [IC_INFLUX_COVID] AS [INFLUX_COVID_PATIENTS],
    dbo.CONVERT_DATETIME_TO_UNIX(DATE_LAST_INSERTED)  AS DATE_OF_INSERTION_UNIX
FROM [VWSDEST].[HOSPITAL_BED_OCCUPANCY]
WHERE DATE_LAST_INSERTED = (SELECT MAX(DATE_LAST_INSERTED)
                            FROM [VWSDEST].[HOSPITAL_BED_OCCUPANCY]);
GO


In [ ]:
CREATE OR ALTER VIEW [VWSDEST].[V_DIFFERENCE_INTENSIVE_CARE_BEDS_OCCUPIED__COVID_OCCUPIED] AS

WITH LAST_DATE_OF_REPORT AS (
	SELECT
		CAST(MAX(DATE_OF_REPORT) as datetime) AS [LAST_DATE_OF_REPORT]
	FROM [VWSDEST].[HOSPITAL_BED_OCCUPANCY]
	WHERE DATE_OF_REPORT >=  '2020-03-16 00:00:00.000'
		AND DATE_LAST_INSERTED = (SELECT MAX(DATE_LAST_INSERTED) FROM [VWSDEST].[HOSPITAL_BED_OCCUPANCY])
),

BASE_CTE AS (
SELECT 
	DATE_OF_REPORT,
	DATE_OF_REPORT_UNIX AS [NEW_DATE_OF_REPORT_UNIX],
	LAG([DATE_OF_REPORT_UNIX], 1, NULL) OVER (ORDER BY [DATE_LAST_INSERTED], [DATE_OF_REPORT_UNIX]) AS [OLD_DATE_OF_REPORT_UNIX],
	LAG([IC_BEDS_OCCUPIED_COVID], 1, NULL) OVER (ORDER BY [DATE_LAST_INSERTED], [DATE_OF_REPORT_UNIX]) AS [OLD_VALUE],
	[IC_BEDS_OCCUPIED_COVID] -
		LAG([IC_BEDS_OCCUPIED_COVID], 1, NULL) OVER (ORDER BY [DATE_LAST_INSERTED], [DATE_OF_REPORT_UNIX]) AS [DIFFERENCE]
--  dbo.CONVERT_DATETIME_TO_UNIX(DATE_LAST_INSERTED) AS DATE_OF_INSERTION_UNIX
FROM [VWSDEST].[HOSPITAL_BED_OCCUPANCY]
WHERE DATE_OF_REPORT >=  '2020-03-02 00:00:00.000'
AND DATE_LAST_INSERTED = (SELECT max(DATE_LAST_INSERTED) 
                            FROM [VWSDEST].[HOSPITAL_BED_OCCUPANCY])

)

SELECT DATE_OF_REPORT
	  ,NEW_DATE_OF_REPORT_UNIX AS NEW_DATE_UNIX
	  ,OLD_DATE_OF_REPORT_UNIX AS OLD_DATE_UNIX
	  ,CASE WHEN OLD_VALUE IS NULL THEN 0 ELSE OLD_VALUE END AS OLD_VALUE
	  ,CASE WHEN [DIFFERENCE] IS NULL THEN 0 ELSE [DIFFERENCE] END AS [DIFFERENCE]
FROM BASE_CTE T1
WHERE DATE_OF_REPORT = (SELECT LAST_DATE_OF_REPORT FROM LAST_DATE_OF_REPORT)
GO


# **DATATINO CONFIGURATIONS**

---

### Sources

In [ ]:
-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN   VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATIONFOR MORE INFORMATION.

-- 1) UPSERT WORKFLOW(S).....
DECLARE @workflow_name NVARCHAR(50) = 'LNAZ_HOSPITAL_BED_OCCUPANCY',
        @workflow_id INT,        
        @workflow_description VARCHAR(256),
        @is_active INT;

-- 1.1) SET ENVIRONMENTAL VARIABLES.....
SET @is_active = CASE LOWER('#{ Environment }#')
    WHEN 'production' THEN 1
    WHEN 'acceptance' THEN 1
    ELSE 1
END;

SET @workflow_id = NULL;

SELECT TOP(1)
    @workflow_id = workflows.[ID]
FROM [DATATINO_ORCHESTRATOR_1].[WORKFLOWS] workflows
INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_WORKFLOWS] v_workflows ON v_workflows.[DATAFLOW_ID] = workflows.[DATAFLOW_ID] AND v_workflows.[ID] = workflows.[ID]
WHERE v_workflows.[NAME] = @workflow_name;

SET @workflow_description = CONCAT('WORKFLOW: ', @workflow_name);

EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_WORKFLOW]
    @id = @workflow_id, 
    @workflow_name = @workflow_name,
    @description = @workflow_description,
    @schedule = '0 14,15 * * MON-FRI',
    @active = @is_active;

-- 2) UPSERT SOURCE(S).....
DECLARE @source NVARCHAR(256),
        @source_id INT,
        @source_name NVARCHAR(256),
        @source_description NVARCHAR(256),
        @target_name VARCHAR(256),
        @location_type VARCHAR(50),
        @security_profile VARCHAR(50);

-- 2.1) SET ENVIRONMENTAL VARIABLES.....
SET @source = CASE LOWER('#{ Environment }#')
    WHEN 'production' THEN 'datafiles/VWS/covid-19-datafeed-oud-v1.csv'
    WHEN 'acceptance' THEN 'datafiles/VWS/covid-19-datafeed-oud-v1.csv'
    ELSE 'datafiles/VWS/covid-19-datafeed-oud-v1.csv'
END;

SET @location_type = CASE LOWER('#{ Environment }#')
    WHEN 'production' THEN 'AzureBlob'
    WHEN 'acceptance' THEN 'AzureBlob'
    ELSE 'AzureBlob'
END;

SET @security_profile = CASE LOWER('#{ Environment }#')
    WHEN 'production' THEN 'N/A'
    WHEN 'acceptance' THEN 'N/A'
    ELSE 'N/A'
END;

-- 2.2) UPSERT TABLE SOURCE(S): STAGING......
SET @source_id = NULL;
SET @source_name = 'SOURCE_Load_Hospital_Bed_Occupancy';
SET @source_description = 'Loads bed occupancy data into target table';
SET @target_name = CONCAT('VWSSTAGE.', @workflow_name);

SELECT TOP(1)
    @source_id = [ID]
FROM [DATATINO_ORCHESTRATOR_1].[SOURCES]
WHERE [NAME] = @source_name;

EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_SOURCE]
    @id = @source_id,
    @source_name = @source_name,
    @description = @source_description,
    @source = @source,
    @source_columns = 'Datum|IC_Bedden_COVID_Nederland|IC_Bedden_Non_COVID_Nederland|Kliniek_Bedden_Nederland|IC_Nieuwe_Opnames_COVID_Nederland|Kliniek_Nieuwe_Opnames_COVID_Nederland',
    @target_columns = 'DATUM|IC_BEDDEN_COVID|IC_BEDDEN_NON_COVID|KLINIEK_BEDDEN|IC_NIEUWE_OPNAMES_COVID|KLINIEK_NIEUWE_OPNAMES_COVID|DATE_LAST_INSERTED=GETDATE',
    @target_name = @target_name,
    @source_type = 'CsvFile',
    @location_type = @location_type,
    @delimiter_type = 'Colon',
    @security_profile= @security_profile;

-- 2.3) UPSERT STORED PROCEDURE SOURCE(S): STAGE TO INTER......
SET @source_id = NULL;
SET @source = 'DBO.SP_LNAZ_HOSPITAL_BED_OCCUPANCY_INTER';
SET @source_name = 'SOURCE_Inter_Hospital_Bed_Occupancy';
SET @source_description = 'Move bed occupancy data from stage table to intermediate table';

SELECT TOP(1)
    @source_id = [ID]
FROM [DATATINO_ORCHESTRATOR_1].[SOURCES]
WHERE [NAME] = @source_name;

EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_SOURCE]
    @id = @source_id,
    @source_name = @source_name,
    @description = @source_description,
    @source = @source,
    @source_columns = null,
    @target_columns = null,
    @target_name = null,
    @source_type = 'StoredProcedure',
    @location_type = 'N/A',
    @delimiter_type = 'N/A',
    @security_profile= @security_profile;

-- 2.4) UPSERT STORED PROCEDURE SOURCE(S): INTER TO DEST.....
SET @source_id = NULL;
SET @source = 'DBO.SP_HOSPITAL_BED_OCCUPANCY';
SET @source_name = 'SOURCE_Dest_Hospital_Bed_Occupancy';
SET @source_description = 'Move bed occupancy data from intermediate table to dest table';

SELECT TOP(1)
    @source_id = [ID]
FROM [DATATINO_ORCHESTRATOR_1].[SOURCES]
WHERE [NAME] = @source_name;

EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_SOURCE]
    @id = @source_id,
    @source_name = @source_name,
    @description = @source_description,
    @source = @source,
    @source_columns = null,
    @target_columns = null,
    @target_name = null,
    @source_type = 'StoredProcedure',
    @location_type = 'N/A',
    @delimiter_type = 'N/A',
    @security_profile= @security_profile;

-- 3) UPSERT PROCESS(ES).....
DECLARE @process_id INT,
        @process_name NVARCHAR(256),
        @process_description NVARCHAR(256),
        @process_source_name VARCHAR(256);

-- 2.1) UPSERT TABLE PROCESS(ES): STAGING......
SET @process_id = NULL;
SET @process_name = 'PROCESS_Load_Hospital_Bed_Occupancy';
SET @process_description = 'Loads bed occupancy data into target table';
SET @process_source_name = 'SOURCE_Load_Hospital_Bed_Occupancy';

SELECT TOP(1)
    @process_id = processes.[ID]
FROM [DATATINO_ORCHESTRATOR_1].[PROCESSES] processes
INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_WORKFLOWS] v_workflows ON v_workflows.[ID] = processes.[WORKFLOW_ID]
INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_PROCESSES] v_processes ON v_processes.[PROCESS_ID] = processes.[ID]
WHERE v_processes.[PROCESS_NAME] = @process_name 
    AND v_workflows.[NAME] = @workflow_name;

EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_PROCESS]
    @id = @process_id,
    @process_name = @process_name,
    @description = @process_description,
    @source_name = @process_source_name,
    @schedule = '* * * * *',
    @workflow_name = @workflow_name,
    @active = 1;

-- 3.2) UPSERT STORED PROCEDURE PROCESS(S): STAGE TO INTER......
SET @process_id = NULL;
SET @process_name = 'PROCESS_Inter_Hospital_Bed_Occupancy';
SET @process_description = 'Move bed occupancy data from stage table to intermediate table';
SET @process_source_name = 'SOURCE_Inter_Hospital_Bed_Occupancy';

SELECT TOP(1)
    @process_id = processes.[ID]
FROM [DATATINO_ORCHESTRATOR_1].[PROCESSES] processes
INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_WORKFLOWS] v_workflows ON v_workflows.[ID] = processes.[WORKFLOW_ID]
INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_PROCESSES] v_processes ON v_processes.[PROCESS_ID] = processes.[ID]
WHERE v_processes.[PROCESS_NAME] = @process_name 
    AND v_workflows.[NAME] = @workflow_name;

EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_PROCESS]
    @id = @process_id,
    @process_name = @process_name,
    @description = @process_description,
    @source_name = @process_source_name,
    @schedule = '* * * * *',
    @workflow_name = @workflow_name,
    @active = 1;

-- 3.3) UPSERT STORED PROCEDURE PROCESS(S): INTER TO DEST......
SET @process_id = NULL;
SET @process_name = 'PROCESS_Dest_Hospital_Bed_Occupancy';
SET @process_description = 'Move bed occupancy data from intermediate table to dest table';
SET @process_source_name = 'SOURCE_Dest_Hospital_Bed_Occupancy';

SELECT TOP(1)
    @process_id = processes.[ID]
FROM [DATATINO_ORCHESTRATOR_1].[PROCESSES] processes
INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_WORKFLOWS] v_workflows ON v_workflows.[ID] = processes.[WORKFLOW_ID]
INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_PROCESSES] v_processes ON v_processes.[PROCESS_ID] = processes.[ID]
WHERE v_processes.[PROCESS_NAME] = @process_name 
    AND v_workflows.[NAME] = @workflow_name;

EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_PROCESS]
    @id = @process_id,
    @process_name = @process_name,
    @description = @process_description,
    @source_name = @process_source_name,
    @schedule = '* * * * *',
    @workflow_name = @workflow_name,
    @active = 1;

-- 4) UPSERT DEPENDENC(Y)(IES).....
DECLARE @dependency_id INT,
        @dependency_name NVARCHAR(256),
        @dependency_description NVARCHAR(256),
        @dependency_dataflow_name NVARCHAR(256),
        @dependency_process_name NVARCHAR(256);

-- 4.1) UPSERT STAGING TO STORE PROCEDURE DEPENDENC(Y)(IES).....
SET @dependency_id = NULL;
SET @dependency_name = 'DEPENDENCY_PROCESS_Inter_Hospital_Bed_Occupancy';
SET @dependency_dataflow_name = 'PROCESS_Inter_Hospital_Bed_Occupancy';
SET @dependency_process_name = 'PROCESS_Load_Hospital_Bed_Occupancy';
SET @dependency_description = 'Dependency for PROCESS_Inter_Hospital_Bed_Occupancy';

SELECT TOP(1)
    @dependency_id = dependencies.[ID]
FROM [DATATINO_ORCHESTRATOR_1].[DEPENDENCIES] dependencies
INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_DEPENDENCIES] v_dependencies ON v_dependencies.[ID] = dependencies.[ID]
WHERE dependencies.[NAME] = @dependency_name
    AND v_dependencies.[WORKFLOW_NAME] = @workflow_name;

EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_DEPENDENCY]
    @id = @dependency_id,
    @dataflow_name = @dependency_dataflow_name,
    @dataflowtype_id = 2,
    @dependency_name = @dependency_process_name,
    @dependencytype_id = 2,
    @workflow_name = @workflow_name,
    @name = @dependency_name,
    @description = @dependency_description,
    @active = 1;

-- 4.2) UPSERT STORED PROCEDURE TO STORED PROCEDURE DEPENDENC(Y)(IES)......
SET @dependency_id = NULL;
SET @dependency_name = 'DEPENDENCY_PROCESS_Dest_Hospital_Bed_Occupancy';
SET @dependency_dataflow_name = 'PROCESS_Dest_Hospital_Bed_Occupancy';
SET @dependency_process_name = 'PROCESS_Inter_Hospital_Bed_Occupancy';
SET @dependency_description = 'Dependency for PROCESS_Dest_Hospital_Bed_Occupancy';

SELECT TOP(1)
    @dependency_id = dependencies.[ID]
FROM [DATATINO_ORCHESTRATOR_1].[DEPENDENCIES] dependencies
INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_DEPENDENCIES] v_dependencies ON v_dependencies.[ID] = dependencies.[ID]
WHERE dependencies.[NAME] = @dependency_name
    AND v_dependencies.[WORKFLOW_NAME] = @workflow_name;

EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_DEPENDENCY]
    @id = @dependency_id,
    @dataflow_name = @dependency_dataflow_name,
    @dataflowtype_id = 2,
    @dependency_name = @dependency_process_name,
    @dependencytype_id = 2,
    @workflow_name = @workflow_name,
    @name = @dependency_name,
    @description = @dependency_description,
    @active = 1;
GO

### PROTOS - HOSPITAL

In [ ]:
-- 1) SET ENVIRONMENTAL VARIABLES.....
DECLARE @view_name VARCHAR(256) = 'VWSDEST.V_NON_IC_BED_OCCUPANCY',
        @view_description VARCHAR(256),
        @item_name VARCHAR(256) = 'hospital_lcps_2022',
        @config_description VARCHAR(256),
        @constraint_value VARCHAR(50) = NULL,
        @constraint_key_name VARCHAR(50) = NULL,
        @grouped_key_name VARCHAR(50) = NULL,
        @grouped_last_update_name VARCHAR(50) = NULL,
        @proto_name VARCHAR(50) = 'NL',
        @columns VARCHAR(256) = '*',
        @layout_type_id INT = 1,
        @last_update_name VARCHAR(50) = 'DATE_UNIX',
        @is_active INT;
        
SET @is_active = CASE LOWER('#{ Environment }#')
    WHEN 'production' THEN 0
    WHEN 'acceptance' THEN 0
    ELSE 0
END;

SET @view_description = 'View Configuration for VWSDEST.V_NON_IC_BED_OCCUPANCY';
SET @config_description = 'View Configuration for hospital_lcps';

-- 2) DETERMINE VIEW ID & CONFIGURATION ID
DECLARE @constrained INT,
        @grouped INT,
        @view_id BIGINT,
        @config_id BIGINT;

SET @constrained = CASE 
    WHEN @constraint_key_name IS NULL THEN 0
    ELSE 1
END;
SET @grouped = CASE 
    WHEN @grouped_key_name IS NULL THEN 0
    ELSE 1
END;

DELETE FROM [DATATINO_PROTO_1].[CONFIGURATIONS]
WHERE [ID] IN (
    SELECT
        configs.[ID]
    FROM [DATATINO_PROTO_1].[VIEWS] views
    INNER JOIN [DATATINO_PROTO_1].[CONFIGURATIONS] AS configs ON views.[ID] = configs.[VIEW_ID]
        AND configs.[NAME] = @item_name
    INNER JOIN [DATATINO_PROTO_1].[PROTOS] AS protos ON protos.[ID] = configs.[PROTO_ID]
        AND protos.[NAME] = @proto_name
);

SELECT 
    @view_id = [ID]
FROM [DATATINO_PROTO_1].[VIEWS]
WHERE ISNULL([CONSTRAINT_VALUE], 'X') = ISNULL(@constraint_value, 'X')
    AND ISNULL([CONSTRAINT_KEY_NAME], 'X') = ISNULL(@constraint_key_name, 'X')
    AND ISNULL([GROUPED_KEY_NAME], 'X') = ISNULL(@grouped_key_name, 'X')
    AND ISNULL([GROUPED_LAST_UPDATE_NAME], 'X') = ISNULL(@grouped_last_update_name, 'X')
    AND [NAME] = @view_name;

SELECT
    @config_id = configs.[ID]
FROM [DATATINO_PROTO_1].[VIEWS] views
INNER JOIN [DATATINO_PROTO_1].[CONFIGURATIONS] AS configs ON views.[ID] = configs.[VIEW_ID]
    AND configs.[NAME] = @item_name
    AND configs.[VIEW_ID] = @view_id
INNER JOIN [DATATINO_PROTO_1].[PROTOS] AS protos ON protos.[ID] = configs.[PROTO_ID]
    AND protos.[NAME] = @proto_name
WHERE views.[NAME] = @view_name;

-- 3) UPSERT PROTO VIEW(S).....
EXECUTE [DATATINO_PROTO_1].[UPSERT_VIEW]
    @id = @view_id,
    @view_name = @view_name,
    @description = @view_description,
    @last_update_name = @last_update_name,
    @constraint_key_name = @constraint_key_name,
    @constraint_value = @constraint_value,
    @grouped_key_name = @grouped_key_name,
    @grouped_last_update_name = @grouped_last_update_name;

-- 4) UPSERT PROTO CONFIGURATION(S).....
EXECUTE [DATATINO_PROTO_1].[UPSERT_CONFIGURATION]
    @id = @config_id,
    @proto_name = @proto_name,
    @description =  @config_description,
    @view_name = @view_name,
    @item_name = @item_name,
    @constrained = @constrained,
    @grouped = @grouped,
    @columns = @columns,
    @mapping = '=LOWER()',
    @layout_type_id = @layout_type_id,
    @active = @is_active,
    @constraint_key_name = @constraint_key_name,
    @constraint_value = @constraint_value,
    @grouped_key_name = @grouped_key_name,
    @grouped_last_update_name = @grouped_last_update_name;
GO

In [ ]:
-- 1) SET ENVIRONMENTAL VARIABLES.....
DECLARE @view_name VARCHAR(256) = 'VWSDEST.V_DIFFERENCE_HOSPITAL_BEDS_OCCUPIED__COVID_OCCUPIED',
        @view_description VARCHAR(256),
        @item_name VARCHAR(256) = 'difference/hospital_lcps__beds_occupied_covid_2022',
        @config_description VARCHAR(256),
        @constraint_value VARCHAR(50) = NULL,
        @constraint_key_name VARCHAR(50) = NULL,
        @grouped_key_name VARCHAR(50) = NULL,
        @grouped_last_update_name VARCHAR(50) = NULL,
        @proto_name VARCHAR(50) = 'NL',
        @columns VARCHAR(256) = 'OLD_VALUE|DIFFERENCE|OLD_DATE_UNIX|NEW_DATE_UNIX',
        @layout_type_id INT = 3, -- LASTVALUES
        @last_update_name VARCHAR(50) = 'NEW_DATE_UNIX',
        @is_active INT;
        
SET @is_active = CASE LOWER('#{ Environment }#')
    WHEN 'production' THEN 0
    WHEN 'acceptance' THEN 0
    ELSE 0
END;

SET @view_description = 'View Configuration for VWSDEST.V_DIFFERENCE_HOSPITAL_BEDS_OCCUPIED__COVID_OCCUPIED';
SET @config_description = 'View Configuration for difference/hospital_lcps__beds_occupied_covid';

-- 2) DETERMINE VIEW ID & CONFIGURATION ID
DECLARE @constrained INT,
        @grouped INT,
        @view_id BIGINT,
        @config_id BIGINT;

SET @constrained = CASE 
    WHEN @constraint_key_name IS NULL THEN 0
    ELSE 1
END;
SET @grouped = CASE 
    WHEN @grouped_key_name IS NULL THEN 0
    ELSE 1
END;

DELETE FROM [DATATINO_PROTO_1].[CONFIGURATIONS]
WHERE [ID] IN (
    SELECT
        configs.[ID]
    FROM [DATATINO_PROTO_1].[VIEWS] views
    INNER JOIN [DATATINO_PROTO_1].[CONFIGURATIONS] AS configs ON views.[ID] = configs.[VIEW_ID]
        AND configs.[NAME] = @item_name
    INNER JOIN [DATATINO_PROTO_1].[PROTOS] AS protos ON protos.[ID] = configs.[PROTO_ID]
        AND protos.[NAME] = @proto_name
);

SELECT 
    @view_id = [ID]
FROM [DATATINO_PROTO_1].[VIEWS]
WHERE ISNULL([CONSTRAINT_VALUE], 'X') = ISNULL(@constraint_value, 'X')
    AND ISNULL([CONSTRAINT_KEY_NAME], 'X') = ISNULL(@constraint_key_name, 'X')
    AND ISNULL([GROUPED_KEY_NAME], 'X') = ISNULL(@grouped_key_name, 'X')
    AND ISNULL([GROUPED_LAST_UPDATE_NAME], 'X') = ISNULL(@grouped_last_update_name, 'X')
    AND [NAME] = @view_name;

SELECT
    @config_id = configs.[ID]
FROM [DATATINO_PROTO_1].[VIEWS] views
INNER JOIN [DATATINO_PROTO_1].[CONFIGURATIONS] AS configs ON views.[ID] = configs.[VIEW_ID]
    AND configs.[NAME] = @item_name
    AND configs.[VIEW_ID] = @view_id
INNER JOIN [DATATINO_PROTO_1].[PROTOS] AS protos ON protos.[ID] = configs.[PROTO_ID]
    AND protos.[NAME] = @proto_name
WHERE views.[NAME] = @view_name;

-- 3) UPSERT PROTO VIEW(S).....
EXECUTE [DATATINO_PROTO_1].[UPSERT_VIEW]
    @id = @view_id,
    @view_name = @view_name,
    @description = @view_description,
    @last_update_name = @last_update_name,
    @constraint_key_name = @constraint_key_name,
    @constraint_value = @constraint_value,
    @grouped_key_name = @grouped_key_name,
    @grouped_last_update_name = @grouped_last_update_name;

-- 4) UPSERT PROTO CONFIGURATION(S).....
EXECUTE [DATATINO_PROTO_1].[UPSERT_CONFIGURATION]
    @id = @config_id,
    @proto_name = @proto_name,
    @description =  @config_description,
    @view_name = @view_name,
    @item_name = @item_name,
    @constrained = @constrained,
    @grouped = @grouped,
    @columns = @columns,
    @mapping = '=LOWER()',
    @layout_type_id = @layout_type_id,
    @active = @is_active,
    @constraint_key_name = @constraint_key_name,
    @constraint_value = @constraint_value,
    @grouped_key_name = @grouped_key_name,
    @grouped_last_update_name = @grouped_last_update_name;
GO

### PROTOS - IC

In [ ]:
-- 1) SET ENVIRONMENTAL VARIABLES.....
DECLARE @view_name VARCHAR(256) = 'VWSDEST.V_IC_BED_OCCUPANCY',
        @view_description VARCHAR(256),
        @item_name VARCHAR(256) = 'intensive_care_lcps_2022',
        @config_description VARCHAR(256),
        @constraint_value VARCHAR(50) = NULL,
        @constraint_key_name VARCHAR(50) = NULL,
        @grouped_key_name VARCHAR(50) = NULL,
        @grouped_last_update_name VARCHAR(50) = NULL,
        @proto_name VARCHAR(50) = 'NL',
        @columns VARCHAR(256) = '*',
        @layout_type_id INT = 1,
        @last_update_name VARCHAR(50) = 'DATE_UNIX',
        @is_active INT;
        
SET @is_active = CASE LOWER('#{ Environment }#')
    WHEN 'production' THEN 0
    WHEN 'acceptance' THEN 0
    ELSE 0
END;

SET @view_description = 'View Configuration for VWSDEST.V_IC_BED_OCCUPANCY';
SET @config_description = 'View Configuration for intensive_care_lcps';

-- 2) DETERMINE VIEW ID & CONFIGURATION ID
DECLARE @constrained INT,
        @grouped INT,
        @view_id BIGINT,
        @config_id BIGINT;

SET @constrained = CASE 
    WHEN @constraint_key_name IS NULL THEN 0
    ELSE 1
END;
SET @grouped = CASE 
    WHEN @grouped_key_name IS NULL THEN 0
    ELSE 1
END;

DELETE FROM [DATATINO_PROTO_1].[CONFIGURATIONS]
WHERE [ID] IN (
    SELECT
        configs.[ID]
    FROM [DATATINO_PROTO_1].[VIEWS] views
    INNER JOIN [DATATINO_PROTO_1].[CONFIGURATIONS] AS configs ON views.[ID] = configs.[VIEW_ID]
        AND configs.[NAME] = @item_name
    INNER JOIN [DATATINO_PROTO_1].[PROTOS] AS protos ON protos.[ID] = configs.[PROTO_ID]
        AND protos.[NAME] = @proto_name
);

SELECT 
    @view_id = [ID]
FROM [DATATINO_PROTO_1].[VIEWS]
WHERE ISNULL([CONSTRAINT_VALUE], 'X') = ISNULL(@constraint_value, 'X')
    AND ISNULL([CONSTRAINT_KEY_NAME], 'X') = ISNULL(@constraint_key_name, 'X')
    AND ISNULL([GROUPED_KEY_NAME], 'X') = ISNULL(@grouped_key_name, 'X')
    AND ISNULL([GROUPED_LAST_UPDATE_NAME], 'X') = ISNULL(@grouped_last_update_name, 'X')
    AND [NAME] = @view_name;

SELECT
    @config_id = configs.[ID]
FROM [DATATINO_PROTO_1].[VIEWS] views
INNER JOIN [DATATINO_PROTO_1].[CONFIGURATIONS] AS configs ON views.[ID] = configs.[VIEW_ID]
    AND configs.[NAME] = @item_name
    AND configs.[VIEW_ID] = @view_id
INNER JOIN [DATATINO_PROTO_1].[PROTOS] AS protos ON protos.[ID] = configs.[PROTO_ID]
    AND protos.[NAME] = @proto_name
WHERE views.[NAME] = @view_name;

-- 3) UPSERT PROTO VIEW(S).....
EXECUTE [DATATINO_PROTO_1].[UPSERT_VIEW]
    @id = @view_id,
    @view_name = @view_name,
    @description = @view_description,
    @last_update_name = @last_update_name,
    @constraint_key_name = @constraint_key_name,
    @constraint_value = @constraint_value,
    @grouped_key_name = @grouped_key_name,
    @grouped_last_update_name = @grouped_last_update_name;

-- 4) UPSERT PROTO CONFIGURATION(S).....
EXECUTE [DATATINO_PROTO_1].[UPSERT_CONFIGURATION]
    @id = @config_id,
    @proto_name = @proto_name,
    @description =  @config_description,
    @view_name = @view_name,
    @item_name = @item_name,
    @constrained = @constrained,
    @grouped = @grouped,
    @columns = @columns,
    @mapping = '=LOWER()',
    @layout_type_id = @layout_type_id,
    @active = @is_active,
    @constraint_key_name = @constraint_key_name,
    @constraint_value = @constraint_value,
    @grouped_key_name = @grouped_key_name,
    @grouped_last_update_name = @grouped_last_update_name;
GO

In [ ]:
-- 1) SET ENVIRONMENTAL VARIABLES.....
DECLARE @view_name VARCHAR(256) = 'VWSDEST.V_DIFFERENCE_INTENSIVE_CARE_BEDS_OCCUPIED__COVID_OCCUPIED',
        @view_description VARCHAR(256),
        @item_name VARCHAR(256) = 'difference/intensive_care_lcps__beds_occupied_covid_2022',
        @config_description VARCHAR(256),
        @constraint_value VARCHAR(50) = NULL,
        @constraint_key_name VARCHAR(50) = NULL,
        @grouped_key_name VARCHAR(50) = NULL,
        @grouped_last_update_name VARCHAR(50) = NULL,
        @proto_name VARCHAR(50) = 'NL',
        @columns VARCHAR(256) = 'OLD_VALUE|DIFFERENCE|OLD_DATE_UNIX|NEW_DATE_UNIX',
        @layout_type_id INT = 3, -- LASTVALUES
        @last_update_name VARCHAR(50) = 'NEW_DATE_UNIX',
        @is_active INT;
        
SET @is_active = CASE LOWER('#{ Environment }#')
    WHEN 'production' THEN 0
    WHEN 'acceptance' THEN 0
    ELSE 0
END;

SET @view_description = 'View Configuration for VWSDEST.V_DIFFERENCE_INTENSIVE_CARE_BEDS_OCCUPIED__COVID_OCCUPIED';
SET @config_description = 'View Configuration for difference/intensive_care_lcps__beds_occupied_covid';

-- 2) DETERMINE VIEW ID & CONFIGURATION ID
DECLARE @constrained INT,
        @grouped INT,
        @view_id BIGINT,
        @config_id BIGINT;

SET @constrained = CASE 
    WHEN @constraint_key_name IS NULL THEN 0
    ELSE 1
END;
SET @grouped = CASE 
    WHEN @grouped_key_name IS NULL THEN 0
    ELSE 1
END;

DELETE FROM [DATATINO_PROTO_1].[CONFIGURATIONS]
WHERE [ID] IN (
    SELECT
        configs.[ID]
    FROM [DATATINO_PROTO_1].[VIEWS] views
    INNER JOIN [DATATINO_PROTO_1].[CONFIGURATIONS] AS configs ON views.[ID] = configs.[VIEW_ID]
        AND configs.[NAME] = @item_name
    INNER JOIN [DATATINO_PROTO_1].[PROTOS] AS protos ON protos.[ID] = configs.[PROTO_ID]
        AND protos.[NAME] = @proto_name
);

SELECT 
    @view_id = [ID]
FROM [DATATINO_PROTO_1].[VIEWS]
WHERE ISNULL([CONSTRAINT_VALUE], 'X') = ISNULL(@constraint_value, 'X')
    AND ISNULL([CONSTRAINT_KEY_NAME], 'X') = ISNULL(@constraint_key_name, 'X')
    AND ISNULL([GROUPED_KEY_NAME], 'X') = ISNULL(@grouped_key_name, 'X')
    AND ISNULL([GROUPED_LAST_UPDATE_NAME], 'X') = ISNULL(@grouped_last_update_name, 'X')
    AND [NAME] = @view_name;

SELECT
    @config_id = configs.[ID]
FROM [DATATINO_PROTO_1].[VIEWS] views
INNER JOIN [DATATINO_PROTO_1].[CONFIGURATIONS] AS configs ON views.[ID] = configs.[VIEW_ID]
    AND configs.[NAME] = @item_name
    AND configs.[VIEW_ID] = @view_id
INNER JOIN [DATATINO_PROTO_1].[PROTOS] AS protos ON protos.[ID] = configs.[PROTO_ID]
    AND protos.[NAME] = @proto_name
WHERE views.[NAME] = @view_name;

-- 3) UPSERT PROTO VIEW(S).....
EXECUTE [DATATINO_PROTO_1].[UPSERT_VIEW]
    @id = @view_id,
    @view_name = @view_name,
    @description = @view_description,
    @last_update_name = @last_update_name,
    @constraint_key_name = @constraint_key_name,
    @constraint_value = @constraint_value,
    @grouped_key_name = @grouped_key_name,
    @grouped_last_update_name = @grouped_last_update_name;

-- 4) UPSERT PROTO CONFIGURATION(S).....
EXECUTE [DATATINO_PROTO_1].[UPSERT_CONFIGURATION]
    @id = @config_id,
    @proto_name = @proto_name,
    @description =  @config_description,
    @view_name = @view_name,
    @item_name = @item_name,
    @constrained = @constrained,
    @grouped = @grouped,
    @columns = @columns,
    @mapping = '=LOWER()',
    @layout_type_id = @layout_type_id,
    @active = @is_active,
    @constraint_key_name = @constraint_key_name,
    @constraint_value = @constraint_value,
    @grouped_key_name = @grouped_key_name,
    @grouped_last_update_name = @grouped_last_update_name;
GO